# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118310e+02     1.466747e+00
 * time: 0.36330509185791016
     1     1.105076e+01     9.337938e-01
 * time: 2.2746291160583496
     2    -1.170333e+01     9.608353e-01
 * time: 2.9344210624694824
     3    -3.374647e+01     7.183843e-01
 * time: 3.917030096054077
     4    -4.706134e+01     5.194214e-01
 * time: 4.905336141586304
     5    -5.669538e+01     2.339190e-01
 * time: 5.868128061294556
     6    -5.976327e+01     1.448905e-01
 * time: 6.521826982498169
     7    -6.090004e+01     4.826378e-02
 * time: 7.186225175857544
     8    -6.130097e+01     8.035715e-02
 * time: 7.839219093322754
     9    -6.159824e+01     3.934706e-02
 * time: 8.492067098617554
    10    -6.182676e+01     3.080881e-02
 * time: 9.146795988082886
    11    -6.201294e+01     2.216044e-02
 * time: 9.802836179733276
    12    -6.207932e+01     2.198535e-02
 * time: 10.456377983093262
    13    -6.214697e+01     1.522257e-02
 * time: 11.10706615447998
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671070
    AtomicLocal         -18.8557713
    AtomicNonlocal      14.8522673
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485382 
    Xc                  -19.3336824

    total               -62.261666459109
